pynq.Overlay.ip_dict()
    ip_dict() shows the information about the IPs used in the designs and will show type(helps to create drivers) and physical address of IPs and address range of IPs.


In [ ]:
import sys
import os

import math
import time
import numpy as np
from datetime import datetime
from pynq import DefaultIP
import pynq
from pynq import MMIO
from pynq import Overlay
from pynq import Xlnk
ol = Overlay('/home/xilinx/jupyter_notebooks/Praeklima /testing/openhab_controls.bit')

# ol.ip_dict

ol.ip_dict to fetch specific information of an IP

MMIO: Memory mapped IO helps to read and write into specific address 
DMA: 
    The pynq DMA class supports the AXI Direct Memory Access IP. This allows data to be read from DRAM, and sent to an AXI stream, or received from a stream and written to DRAM
    
Xlnk: 
    Memory must be allocated before it can be accessed by the IP. Xlnk allows memory buffers to be allocated. Xlnk allocates a contiguous memory buffer which allows efficient transfers of data between PS and PL. Python or other code running in Linux on the PS can access the memory buffer directly.
    
   for more info and functions on these class
   https://pynq.readthedocs.io/en/v2.0/overlay_design_methodology/pspl_interface.html#

In [2]:
dp_32x1k_byte_addr0 = ol.ip_dict['axi_bram_ctrl_0']['phys_addr']
print('User space memory BYTE address of IP block dual port ram: ' + str(hex(dp_32x1k_byte_addr0)))
dp_32x1k_byte_size0 = ol.ip_dict['axi_bram_ctrl_0']['addr_range']
print('Size in BYTES of memory mapped ip block: ' + str((dp_32x1k_byte_size0)));
dp_32x1k_byte_addr1 = ol.ip_dict['axi_bram_ctrl_1']['phys_addr']
print('User space memory BYTE address of IP block dual port ram: ' + str(hex(dp_32x1k_byte_addr1)))
dp_32x1k_byte_size1 = ol.ip_dict['axi_bram_ctrl_1']['addr_range']
print('Size in BYTES of memory mapped ip block: ' + str((dp_32x1k_byte_size1)));

dpram1 = MMIO(dp_32x1k_byte_addr0, dp_32x1k_byte_size0)
dpram2 = MMIO(dp_32x1k_byte_addr1, dp_32x1k_byte_size1)


xlnk = pynq.Xlnk()
in_buffer = xlnk.cma_array(shape=(20,), dtype=np.uint32)
out_buffer = xlnk.cma_array(shape=(20,), dtype=np.uint32)

dma1 = ol.axi_dma_0
start = ol.axi_gpio_0.channel1 # AXI GPIO has two functions read and write helps to read and write from GPIOs.
done = ol.axi_gpio_0.channel2
idle = ol.axi_gpio_1.channel1
ready = ol.axi_gpio_1.channel2

User space memory BYTE address of IP block dual port ram: 0xa0000000
Size in BYTES of memory mapped ip block: 8192
User space memory BYTE address of IP block dual port ram: 0xa0002000
Size in BYTES of memory mapped ip block: 8192


/usr/lib/python3/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: pynq.Xlnk is deprecated and will be removed in 2.7 - use pynq.allocate instead
  
/usr/local/lib/python3.6/dist-packages/pynq/overlay.py:691: UserWarning: Interrupt s2mm_introut not created: Could not find UIO device for interrupt pin for IRQ number 122
  interrupt, str(e)))


functions to write and read a whole list into memory (BRAM)
dma_transfer() Transmitts and receives stream of data into DMA FIFO.

In [3]:


def write32_list(mem, idx, val):
    for i in range(len(idx)):
        mem.write(idx[i] * 4, val[i])
def write32(mem, idx, val):
    mem.write(idx*4, val)
def write_bytes(mem, start_idx, vals):
    mem.write(start_idx*4, vals)
def read32_list(mem, idx):
    data = []
    for i in idx:
        data.append(mem.read(i*4))
    return data
def read32(mem, idx):
    return mem.read(idx*4)

def dma_transfer(dma):
    dma.sendchannel.transfer(in_buffer)
    dma.recvchannel.transfer(out_buffer)    
    dma.sendchannel.wait()
    dma.recvchannel.wait()

import requests
import pandas
import time
import numpy
from datetime import datetime
from fxpmath import Fxp

database = pandas.read_csv('/home/xilinx/jupyter_notebooks/Praeklima /REST_API_database')  # Opening csv file using pandas

# Column names for the database
column = ['date', 'time', 'localCurrentTemperature', 'localCurrentApparentTemperature', 'LocalForecastedTemperature_3',
          'LocalForecastedApparentTemperature_3', 'LocalForecastedTemperature_6',
          'LocalForecastedApparentTemperature_6', 'localCurrentHumidity', 'LocalForecastedHumidity_3',
          'LocalForecastedHumidity_6', 'LocalCurrent_Cloudiness', 'LocalForecastedCloudiness_3',
          'LocalForecastedCloudiness_6', 'localCurrentUVIndex', 'LocalCurrent_Rain', 'LocalForecastedRain_3',
          'LocalForecastedRain_6',
          'LocalCurrent_Snow', 'LocalForecastedSnow_3', 'LocalForecastedSnow_6', 'SensorTemperature',
          'SensorRelativeHumidity', 'SensorUltraviolet', 'SensorLuminance', 'MotionAlarm', 'TamperAlarm',
          'WallPlugSwitch_Switch_1', 'WallPlugSwitch_SensorPower_1']
row_data = []
listdata = database.values.tolist()

y = [1]
for i in range(10):
    y.append((Fxp(listdata[1][i+3], signed=True, n_word=32, n_frac=16)).bin(frac_dot = True))
print(y)

help(pynq.Xlnk) helps to find the functions syntax offered by specific class 

In [16]:
addr = [0,1,2,3,4,5,6,7,8,9] #address locations of BRAM
sensor_values = [0,0,0,0,0,0,0,0,0,0] 
for i in range(10):
    sensor_values[i] = 70
for i in range(20):
    in_buffer[i] = (i%10)*10
write32_list(dpram1,addr,sensor_values) # writing an array into BRAM_1
print(read32_list(dpram1,addr))


[70, 70, 70, 70, 70, 70, 70, 70, 70, 70]


In [17]:
start.write(1,0x1) # starting IP by activating ap_start pin 



In [18]:

# dma.running
dma_transfer(dma1)

In [19]:
for i in range(100):
    if(ready.read == 1):
        print(ready.read() + "ready is active")
    if (done.read()== 1):
        print("execution completed")
        start.write(0,0x1)

In [20]:
print(read32_list(dpram1,addr))
print(read32_list(dpram2,addr)) # reading output 

[70, 70, 70, 70, 70, 70, 70, 70, 70, 70]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]


In [21]:
start.write(0,0x1) #stoping the IP 
print(read32_list(dpram1,addr))
print(read32_list(dpram2,addr))
#print the output stream
print(out_buffer)

[70, 70, 70, 70, 70, 70, 70, 70, 70, 70]
[1, 1, 1, 1, 1, 1, 1, 0, 0, 0]
[1 1 1 1 1 1 1 0 0 0 1 1 1 1 1 1 1 0 0 0]
